In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!mkdir -p dataset/images predictions
!wget https://vizwiz.cs.colorado.edu/VizWiz_final/images/train.zip \
      https://vizwiz.cs.colorado.edu/VizWiz_final/images/val.zip \
      https://vizwiz.cs.colorado.edu/VizWiz_final/images/test.zip

--2024-04-16 02:20:20--  https://vizwiz.cs.colorado.edu/VizWiz_final/images/train.zip
Resolving vizwiz.cs.colorado.edu (vizwiz.cs.colorado.edu)... 198.59.7.50
Connecting to vizwiz.cs.colorado.edu (vizwiz.cs.colorado.edu)|198.59.7.50|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11298421598 (11G) [application/zip]
Saving to: ‘train.zip.1’

train.zip.1          66%[============>       ]   7.02G  64.4MB/s    eta 62s    ^C


In [7]:
!unzip -q -o train.zip -d dataset/images
!unzip -q -o val.zip -d dataset/images
!unzip -q -o test.zip -d dataset/images

In [ ]:
!rm train.zip val.zip test.zip

In [ ]:
!pip3 install virtualenv
!virtualenv "/content/drive/MyDrive/CMSC_472/CMSC 472 Project/Code/virtual_env"
!chmod 755 "/content/drive/MyDrive/CMSC_472/CMSC 472 Project/Code/virtual_env/bin/activate"; pip install timm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 32.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.9/468.9 kB 39.0 MB/s eta 0:00:00
created virtual environment CPython3.10.12.final.0-64 in 11651ms
  creator CPython3Posix(dest=/content/drive/MyDrive/CMSC_472/CMSC 472 Project/Code/virtual_env, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, setuptools=bundle, wheel=bundle, via=copy, app_data_dir=/root/.local/share/virtualenv)
    added seed packages: pip==24.0, setuptools==69.1.0, wheel==0.42.0
  activators BashActivator,CShellActivator,FishActivator,NushellActivator,PowerShellActivator,PythonActivator
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 23.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-m

In [8]:
import os
import argparse
import json
from datetime import datetime

import numpy as np
from PIL import Image

import torch
import torch.nn as nn
from torch.utils.data import Dataset
import torchvision
from torchvision import transforms

import sys
# add the path of the virtual environmentsite-packages to colab system path
sys.path.append("virtual_env/lib/python3.10/site-packages")
import timm

In [ ]:
!pip uninstall gdown -y && pip install gdown
!pip install -U openmim
!mim install mmcv

Found existing installation: gdown 5.1.0
Uninstalling gdown-5.1.0:
  Successfully uninstalled gdown-5.1.0
  Using cached gdown-5.1.0-py3-none-any.whl (17 kB)
Looking in links: https://download.openmmlab.com/mmcv/dist/cu121/torch2.2.0/index.html
  Using cached mmcv-2.1.0.tar.gz (471 kB)
  Preparing metadata (setup.py) ... done
  Using cached addict-2.4.0-py3-none-any.whl (3.8 kB)
  Using cached mmengine-0.10.3-py3-none-any.whl (451 kB)
  Using cached yapf-0.40.2-py3-none-any.whl (254 kB)


In [3]:
import gdown
url = 'https://drive.google.com/file/d/1qh5F6_u4NWnMMlw2q-mNzc4-7y427dsX/view?usp=drive_link'
output = 'all.pth'
gdown.download(url, output, quiet=False)

/usr/local/lib/python3.10/dist-packages/gdown/parse_url.py:48: UserWarning: You specified a Google Drive link that is not the correct link to download a file. You might want to try `--fuzzy` option or the following url: https://drive.google.com/uc?id=1qh5F6_u4NWnMMlw2q-mNzc4-7y427dsX
  warnings.warn(
Downloading...
From: https://drive.google.com/file/d/1qh5F6_u4NWnMMlw2q-mNzc4-7y427dsX/view?usp=drive_link
To: /content/all.pth
82.8kB [00:00, 32.0MB/s]


'all.pth'

In [5]:
!git clone https://github.com/XLearning-SCU/2022-CVPR-AirNet

Cloning into '2022-CVPR-AirNet'...
remote: Enumerating objects: 124, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 124 (delta 4), reused 3 (delta 3), pack-reused 112
Receiving objects: 100% (124/124), 36.54 MiB | 37.98 MiB/s, done.
Resolving deltas: 100% (24/24), done.


In [10]:
!mkdir airnet
!python 2022-CVPR-AirNet/demo.py --test_path='./dataset/images' --output_path='./airnet'

Traceback (most recent call last):
  File "/content/2022-CVPR-AirNet/demo.py", line 12, in <module>
    from net.model import AirNet
  File "/content/2022-CVPR-AirNet/net/model.py", line 4, in <module>
    from net.DGRN import DGRN
  File "/content/2022-CVPR-AirNet/net/DGRN.py", line 2, in <module>
    from .deform_conv import DCN_layer
  File "/content/2022-CVPR-AirNet/net/deform_conv.py", line 7, in <module>
    from mmcv.ops import modulated_deform_conv2d
ModuleNotFoundError: No module named 'mmcv'


In [ ]:
ann_path = 'dataset/annotations.json'
images_path = 'airnet'
prediction_path = 'predictions/'
model_name = 'airnet-resnet50'
batch_size = 64

os.chdir("/content/drive/MyDrive/CMSC_472/CMSC 472 Project/Code/")

In [ ]:
annotations = json.load(open(ann_path))
indices_in_1k = [d['id'] for d in annotations['categories']]

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                          std=[0.229, 0.224, 0.225])
    ])

class VizWizClassification(Dataset):
    def __init__(self, annotations, transform=None):
        self.images = []
        for img in annotations['images']:
          if 'train' in img:
            self.images.append(images_path + '/train/' + str(img))

          if 'val' in img:
            self.images.append(images_path + '/val/' + str(img))

          if 'test' in img:
            self.images.append(images_path + '/test/' + str(img))

        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = Image.open(self.images[idx]).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, self.images[idx].split("/")[3]

dataset = VizWizClassification(annotations,test_transform)
vizwiz_loader = torch.utils.data.DataLoader(dataset,batch_size=batch_size, shuffle=False)

In [ ]:
model = timm.create_model(model_name, pretrained=True).to(device)
model.eval()

In [ ]:
results = {}
with torch.no_grad():
    for images, images_path in vizwiz_loader:
        images = images.to(device)
        outputs = model(images)[:,indices_in_1k]
        pred = list(outputs.data.max(1)[1].cpu())
        for i in range(len(pred)):
                results[images_path[i]] = indices_in_1k[pred[i]]

In [ ]:
file_path = os.path.join(prediction_path, datetime.now().strftime("prediction-%m-%d-%Y-%H:%M:%S.json"))
with open(file_path, 'w') as outfile:
    json.dump(results, outfile)